In [0]:
%run "../includes/configuration"

## Aggregate Functions demo

#### Built-in aggregate functions

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")
display(race_results_df)

In [0]:
from pyspark.sql.functions import count, countDistinct, sum

In [0]:
race_results_df.select(count("*")).show()

In [0]:
race_results_df.select(countDistinct("race_name")).show()

In [0]:
race_results_df.select(sum("points")).show()

In [0]:
race_results_df.filter(race_results_df.driver_name == 'Lewis Hamilton').select(sum("points")).show()

In [0]:
race_results_df.filter(race_results_df.driver_name == 'Lewis Hamilton').select(sum("points").alias("total_points"), countDistinct("race_name").alias("number_of_races")).show()

In [0]:
display(race_results_df.\
                groupBy('driver_name')\
                .sum('points'))

In [0]:
display(race_results_df.\
                groupBy('driver_name')\
                .agg(sum('points').alias("total_points"), countDistinct('race_name').alias('number_of_races')))

In [0]:
demo_df = race_results_df.filter('race_year in (2019, 2020)')
display(demo_df)

In [0]:
demo_grouped_df = demo_df.\
                groupBy('race_year', 'driver_name')\
                .agg(sum('points').alias("total_points"), countDistinct('race_name').alias('number_of_races'))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

driver_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'))
demo_rank_df = demo_grouped_df.withColumn('rank', rank().over(driver_rank_spec))

In [0]:
display(demo_rank_df)